<a href="https://colab.research.google.com/github/owenbean400/AmazonSentimentAnalysis/blob/master/Anne_Turmel_Owen_Bean_Amazon_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Anne Turmel and Owen Bean NLP Final Project Spring 2023

Placeholder for project summary; write when writing abstract for report? 

In [27]:
import torch
torch.manual_seed(1324224321)

In [28]:
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_name = "albert-base-v2"
model_name = "fine_tune_amazon_Toys_LUKE"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)
model_luke = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)

def read_amazon_product_review(file_name):
    df = pd.read_csv(file_name, sep=",", header=0)
    dic_review = {}
    dic_review_LUKE = {}
    df1 = df[["reviewerID", "overall", "reviewText_LUKE", "reviewText"]]
    for index in range(len(df1)):
        rating = df.loc[index, "overall"]
        label = int(rating) - 1
        
        review_id = df.loc[index, "reviewerID"]
        if label in dic_review:
            dic_review[label][review_id] = df.loc[index, "reviewText"]
        else:
            dic_review[label] = {review_id: df.loc[index, "reviewText"]}
        if label in dic_review_LUKE:
            dic_review_LUKE[label][review_id] = df.loc[index, "reviewText_LUKE"]
        else:
            dic_review_LUKE[label] = {review_id: df.loc[index, "reviewText_LUKE"]}

    return dic_review, dic_review_LUKE


def split_data(amazon_data):
    training = {}
    validation = {}
    test = {}

    for label in amazon_data:
        temp_dic = amazon_data[label]
        lst_amazon_ids = list(temp_dic.keys())
        train_length = int(len(lst_amazon_ids) * 0.8)
        train_ids = lst_amazon_ids[:train_length]
        remaining = lst_amazon_ids[train_length:]
        test_lenght = int(len(remaining) * 0.5)
        test_ids = remaining[:test_lenght]
        validation_id = remaining[test_lenght:]

        for tweet_id in train_ids:
            training[temp_dic[tweet_id]] = label
        for tweet_id in validation_id:
            validation[temp_dic[tweet_id]] = label
        for tweet_id in test_ids:
            test[temp_dic[tweet_id]] = label

    return training, validation, test

dic_review, dic_review_LUKE = read_amazon_product_review("Toys_and_Games copy.csv")
training, validation, test = split_data(dic_review)
training_luke, validation_luke, test_luke = split_data(dic_review)

class AmazonProductReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(list(training.keys()), truncation=True, padding=True)
val_encodings = tokenizer(list(validation.keys()), truncation=True, padding=True)
test_encodings = tokenizer(list(test.keys()), truncation=True, padding=True)
train_encodings_luke = tokenizer(list(training_luke.keys()), truncation=True, padding=True)
val_encodings_luke = tokenizer(list(validation_luke.keys()), truncation=True, padding=True)
test_encodings_luke = tokenizer(list(test_luke.keys()), truncation=True, padding=True)
train_dataset = AmazonProductReviewDataset(train_encodings, list(training.values()))
val_dataset = AmazonProductReviewDataset(val_encodings, list(validation.values()))
test_dataset = AmazonProductReviewDataset(test_encodings, list(test.values()))
train_dataset_luke = AmazonProductReviewDataset(train_encodings_luke, list(training.values()))
val_dataset_luke = AmazonProductReviewDataset(val_encodings_luke, list(validation.values()))
test_dataset_luke = AmazonProductReviewDataset(test_encodings_luke, list(test.values()))



{'Looks just like my "Bad Kitty" "Ms. Rue" which I swear the Author knows. Nick keep up the good work.': 4, 'My son likes playing with Otis the tractor as I read along to the book. a great little stuffed tractor!': 4, 'This was for one of my grandchildren.  He was thrilled with it, so, needless to say, so am I.': 4, 'This is just adorable!  Plan on getting a tattoo of this kitty!': 4, 'LOVE IT': 4, 'Exactly as described. Thank you!': 4, 'Awww.. this was a nice addition to my book character doll collection.': 4, 'My grandson was so very happy!': 4, 'a bedtime favorite': 4, 'Perfect!!': 4, "We got this for my son to accompany the Otis book we bought for him through Amazon. It's adorable and is one of his constant nap companions.": 4, "Plush toy bought for 2-yr old fan of Otis. He's slept with it since!": 4, 'Christmas present': 4, 'My sons loves it. Bigger then I thought. Very nice': 4, 'Cute as can be, goes with the book great!': 4, 'A great companion to the Otis book series, beloved  b

In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np
from datasets import load_metric

model_name = "fine_tune_amazon_Test"
model_name_luke = "fine_tune_amazon_Test"
learning_rate = 2e-5
per_device_train_batch_size = 16
per_device_eval_batch_size = 16
num_train_epochs = 1
save_strategy = "no"

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   return {"accuracy": accuracy}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(
   output_dir=model_name,
   learning_rate=learning_rate,
   per_device_train_batch_size=per_device_train_batch_size,
   per_device_eval_batch_size=per_device_eval_batch_size,
   num_train_epochs=num_train_epochs,
   save_strategy = save_strategy
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

training_args_luke = TrainingArguments(
   output_dir=model_name_luke,
   learning_rate=learning_rate,
   per_device_train_batch_size=per_device_train_batch_size,
   per_device_eval_batch_size=per_device_eval_batch_size,
   num_train_epochs=num_train_epochs,
   save_strategy = save_strategy
)

trainer_luke = Trainer(
   model=model_luke,
   args=training_args,
   train_dataset=train_dataset_luke,
   eval_dataset=val_dataset_luke,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

# print("Training one")
# trainer.train()
# trainer.save_model()
# print(trainer.evaluate())
# Test dataset
trainer.eval_dataset = test_dataset

print("Testing one")
# print(trainer.evaluate())
# or
trainer.predict(test_dataset)

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Testing one
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/58 [00:00<?, ?it/s]

/var/folders/ng/15hc6jqj3r56lj_4ytdc6_5r0000gn/T/ipykernel_1098/4287718247.py:16: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


PredictionOutput(predictions=array([[-2.5507386 , -2.7771673 , -0.0986963 ,  2.3222296 ,  2.4437382 ],
       [-1.927534  , -3.3783298 , -1.5689471 ,  2.0027003 ,  5.04352   ],
       [-1.8806508 , -3.1563885 , -1.7069924 ,  1.397546  ,  5.1144805 ],
       ...,
       [ 1.0667844 ,  1.9439543 ,  0.5843462 , -0.6411721 , -1.9340879 ],
       [-2.4775844 , -2.2789834 ,  0.39335698,  3.26212   ,  0.9961867 ],
       [-0.46653423,  1.3386538 ,  2.1323454 , -0.12913203, -2.119496  ]],
      dtype=float32), label_ids=array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4

In [32]:
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_name = "fine_tune_amazon_Video_Games_Original"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)
model_luke = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)

def read_amazon_product_review(file_name):
    df = pd.read_csv(file_name, sep=",", header=0)
    dic_review = {}
    dic_review_LUKE = {}
    df1 = df[["reviewerID", "overall", "reviewText_LUKE", "reviewText"]]
    for index in range(len(df1)):
        rating = df.loc[index, "overall"]
        label = int(rating) - 1
        
        review_id = df.loc[index, "reviewerID"]
        if label in dic_review:
            dic_review[label][review_id] = df.loc[index, "reviewText"]
        else:
            dic_review[label] = {review_id: df.loc[index, "reviewText"]}
        if label in dic_review_LUKE:
            dic_review_LUKE[label][review_id] = df.loc[index, "reviewText_LUKE"]
        else:
            dic_review_LUKE[label] = {review_id: df.loc[index, "reviewText_LUKE"]}

    return dic_review, dic_review_LUKE

def data_for_testing(amazon_data):
    test = {}

    for label in amazon_data:
        temp_dic = amazon_data[label]
        lst_amazon_ids = list(temp_dic.keys())

        for amazon_id in lst_amazon_ids:
            test[temp_dic[amazon_id]] = label

    # print(test)

    return test

dic_review_2, dic_review_LUKE_2 = read_amazon_product_review("all_beauty.csv")
test = data_for_testing(dic_review_LUKE_2)
test_luke = data_for_testing(dic_review_LUKE_2)

class AmazonProductReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_encodings_2 = tokenizer(list(test.keys()), truncation=True, padding=True)
test_encodings_luke_2 = tokenizer(list(test_luke.keys()), truncation=True, padding=True)
test_dataset = AmazonProductReviewDataset(test_encodings_2, list(test.values()))
test_dataset_luke = AmazonProductReviewDataset(test_encodings_luke_2, list(test.values()))

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

trainer.eval_dataset = test_dataset

print("Testing one")
# print(trainer.evaluate())
# or
trainer.predict(test_dataset)

Testing one


  0%|          | 0/546 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-1.9339409 , -2.0513606 , -1.8594129 ,  0.7517627 ,  5.8318963 ],
       [ 2.956022  ,  1.2627822 , -0.10502013, -1.9346685 , -2.2902296 ],
       [ 1.6197163 ,  1.4315903 ,  0.45298955, -1.8372823 , -2.409861  ],
       ...,
       [-2.750424  , -1.5882747 ,  1.4705243 ,  2.5082624 , -0.6771462 ],
       [ 2.315982  ,  1.7021317 ,  0.5817799 , -2.2741828 , -3.081249  ],
       [-2.530894  , -1.178642  ,  0.94930875,  2.8100815 , -0.7687113 ]],
      dtype=float32), label_ids=array([4, 0, 0, ..., 2, 2, 2]), metrics={'test_loss': 0.6954521536827087, 'test_accuracy': 0.7913372292884152, 'test_runtime': 151.4114, 'test_samples_per_second': 57.638, 'test_steps_per_second': 3.606})